In [16]:
import os
import numpy as np
import torch
from transformers import BertTokenizer, BertTokenizerFast, BertForSequenceClassification, BertModel
import math
import torch.nn as nn
import os
import numpy as np
import re
import matplotlib
import argparse
# matplotlib.use('Qt5Agg')
# import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch import nn
import copy
import math
from tqdm.auto import tqdm
from torch.nn import SyncBatchNorm
from torch.utils.data.dataloader import default_collate
from torch.nn.parallel import DistributedDataParallel
import torch.distributed as dist
from torch.utils.data.distributed import DistributedSampler
# from torch.utils.tensorboard import SummaryWriter
import datasets
import evaluate
from datasets import load_dataset
work_dir = "./"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
dataset = load_dataset("cnn_dailymail", '3.0.0')
dataset

Found cached dataset cnn_dailymail (/home/ldmc/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 595.84it/s]


DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [19]:
dataset["train"][0]["article"]

'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details o

In [6]:
dataset["train"][0]["highlights"]

"Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .\nYoung actor says he has no plans to fritter his cash away .\nRadcliffe's earnings from first five Potter films have been held in trust fund ."

In [22]:
from nltk.tokenize import sent_tokenize
sent_tokenize(dataset["train"][0]["article"])

["LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him.",
 'Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties.',
 '"I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month.',
 '"I don\'t think I\'ll be particularly extravagant.',
 '"The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs."',
 'At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box off

In [27]:
def split_article(example):
    example["split_article"] =sent_tokenize(example["article"])
    example["split_highlights"] =sent_tokenize(example["highlights"])
    return example

In [28]:
dataset = dataset.map(split_article, num_proc=1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 11490/11490 [00:11<00:00, 1004.97ex/s]


In [29]:
dataset["train"][1]["split_article"]

["Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.",
 'Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.',
 'MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."',
 "Here, inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court.",
 'Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies."',
 'He says the arrests often result from confrontations with police.',
 "Mentally ill people often won't do what they're told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, an

In [30]:
dataset["train"][1]["split_highlights"]

['Mentally ill inmates in Miami are housed on the "forgotten floor"\nJudge Steven Leifman says most are there as a result of "avoidable felonies"\nWhile CNN tours facility, patient shouts: "I am the son of the president"\nLeifman says the system is unjust and he\'s fighting for change .']

In [33]:
max_article_len = 0
for split_article in tqdm(dataset["train"]["split_article"]):
    max_article_len = max(max_article_len, len(split_article))
for split_article in tqdm(dataset["validation"]["split_article"]):
    max_article_len = max(max_article_len, len(split_article))
for split_article in tqdm(dataset["test"]["split_article"]):
    max_article_len = max(max_article_len, len(split_article))
max_article_len

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 11490/11490 [00:00<00:00, 1762132.18it/s]


399

In [37]:
rouge = evaluate.load('rouge')
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
results = rouge.compute(predictions=predictions, references=references)
print(results)

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}


In [39]:
rouge = evaluate.load('rouge')
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
results = rouge.compute(predictions=["hello there"], references=["hello there"])
print(results)

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}


In [41]:
def get_extractive_summary(example):
    extractive_summary_label = [0 for _ in range(max_article_len)]
    extractive_summary_positions = set()
    for i, highlights_sent in enumerate(example["split_highlights"]):
        max_rouge_1 = 0
        max_rouge_1_position = None
        for j, article_sent in enumerate(example["split_article"]):
            if j not in extractive_summary_positions:
                rouge_scores = rouge.compute(predictions=[article_sent], references=[highlights_sent])
                if rouge_scores["rouge1"] > max_rouge_1:
                    max_rouge_1 = rouge_scores["rouge1"]
                    max_rouge_1_position = j
        extractive_summary_label[max_rouge_1_position] = 1
        extractive_summary_positions.add(max_rouge_1_position)
    example["extractive_summary_label"] = extractive_summary_label
    return example

In [43]:
dataset = dataset.map(get_extractive_summary, num_proc=16)

#0:   0%|                                                                                                                | 0/17945 [00:00<?, ?ex/s]




#5:   0%|                                                                                                                | 0/17945 [00:00<?, ?ex/s]



#4:   0%|                                                                                                                | 0/17945 [00:00<?, ?ex/s]


#3:   0%|                                                                                                                | 0/17945 [00:00<?, ?ex/s]

#2:   0%|                                                                                                                | 0/17945 [00:00<?, ?ex/s]





#1:   0%|                                                                                                                | 0/17945 [00:00<?, ?ex/s]






#7:   0%|                                                                                  

KeyboardInterrupt: 

In [ ]:
def add_cls_sep_token(example):
    example["src_txt_cls_join"] = " [SEP] [CLS] ".join(example["src_txt"])
    return example

In [ ]:
added_cls_dataset = dataset.map(add_cls_sep_token, num_proc=30)